In [1]:
from randomForestRules import RandomForestRules
import pandas as pd

In [2]:
df = pd.read_csv("data/audiology.csv")
df.columns = df.columns.str.replace("_", "-") # underscore not allowed
df.head()

,age-gt-60,air,airBoneGap,ar-c,ar-u,bone,boneAbnormal,bser,history-buzzing,history-dizziness,...,s-sn-gt-1k,s-sn-gt-2k,s-sn-gt-4k,speech,static-normal,tymp,viith-nerve-signs,wave-V-delayed,waveform-ItoV-prolonged,binaryClass
0,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,P
1,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,P
2,t,normal,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,good,t,a,f,f,f,N
3,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,unmeasured,t,a,f,f,f,N
4,t,mild,f,normal,normal,unmeasured,f,?,f,f,...,f,f,f,normal,t,a,f,f,f,P


In [3]:
cols=[]
for col in df.columns:
    if col != 'binaryClass':
        cols.append(col)

In [4]:
randomForest = RandomForestRules()
randomForest.load_pandas(df)
randomForest.fit(antecedent = cols, 
                 consequent = 'binaryClass', 
                 supp=0.4, 
                 conf=50,
                 random_state = 42
                )
frame = randomForest.get_frame()

In [5]:
print(frame)

     age-gt-60     air airBoneGap      ar-c    ar-u        bone boneAbnormal  \
0          NaN     NaN        NaN    absent     NaN         NaN          NaN   
1          NaN     NaN        NaN  elevated     NaN         NaN          NaN   
2          NaN     NaN        NaN         ?     NaN         NaN          NaN   
3          NaN     NaN        NaN    absent     NaN         NaN          NaN   
4          NaN     NaN        NaN         ?     NaN         NaN          NaN   
...        ...     ...        ...       ...     ...         ...          ...   
1597       NaN    mild        NaN       NaN  normal  unmeasured          NaN   
1598         f  normal        NaN    normal  normal         NaN            f   
1599       NaN    mild        NaN       NaN  normal  unmeasured          NaN   
1600       NaN    mild        NaN       NaN  normal           ?          NaN   
1601         t  normal        NaN    normal  normal         NaN            f   

     bser history-buzzing history-dizzi

In [6]:
from actionrules.actionRulesDiscovery import ActionRulesDiscovery
actionRulesDiscovery = ActionRulesDiscovery()
actionRulesDiscovery.load_pandas(frame)
actionRulesDiscovery.fit_classification_rules(stable_attributes = cols[0:6],
                                 flexible_attributes = cols[6:],
                                 consequent = 'binaryClass',
                                 conf_col = 'confidence',
                                 supp_col = 'support',
                                 desired_classes = ['P'])

In [7]:
len(actionRulesDiscovery.get_action_rules())

7

In [8]:
for rule in actionRulesDiscovery.get_action_rules_representation():
    print(rule)
    print(" ")

r = [(airBoneGap: f) ∧ (ar-c: normal) ∧ (m-sn-gt-4k: f → t)  ∧ (o-ar-c: absent → elevated) ] ⇒ [binaryClass: N → P] with support: 0.004424778761061947, confidence: 0.6666666666666666 and uplift: 0.008849557522123894.
 
r = [(air: mild) ∧ (ar-c: absent) ∧ (speech: normal → good) ] ⇒ [binaryClass: N → P] with support: 0.008849557522123894, confidence: 0.5 and uplift: 0.022123893805309734.
 
r = [(air: mild) ∧ (ar-c: absent) ∧ (speech: very_good → good) ] ⇒ [binaryClass: N → P] with support: 0.008849557522123894, confidence: 0.42857142857142855 and uplift: 0.011061946902654867.
 
r = [(air: mild) ∧ (ar-c: absent) ∧ (speech: very_poor → good) ] ⇒ [binaryClass: N → P] with support: 0.004424778761061947, confidence: 0.5 and uplift: 0.0022123893805309734.
 
r = [(air: mild) ∧ (ar-c: absent) ∧ (speech: ? → good) ] ⇒ [binaryClass: N → P] with support: 0.008849557522123894, confidence: 0.5 and uplift: 0.004424778761061947.
 
r = [(air: mild) ∧ (ar-c: absent) ∧ (speech: poor → good) ] ⇒ [binaryCl